
# <center><font color= #F08080 > <b>CU53_impacto de las políticas de inversión en sanidad, infraestructuras y promoción turística en el SPI</font></center>

#  <font color='green'>IV. Model development</font>

En este anexo se incluye el código utilizado durante el desarrollo 
de los modelos incluidos en el caso de uso.

### Paquetes


In [1]:
library(readr)
library(dplyr)
library(tidyr)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




## Generación de escenarios



In [2]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "es_ES.UTF-8/es_ES.UTF-8/es_ES.UTF-8/C/es_ES.UTF-8/C"

In [3]:
inversiones <- read_csv("CU_53_05_05_inversiones_cm.csv")

spi <- read_csv("CU_53_05_02_01_spi.csv")  
spi_meta <- read_csv("CU_53_05_02_02_spi_metadata.csv")

inversiones <- inversiones |> 
  group_by(anyo) |> 
  mutate(total_anyo = sum(inversion),
         porc_inv = round(100*(inversion/total_anyo), 2)) |> 
  ungroup() 

parametros_sim <- inversiones |> group_by(grupo) |> 
  summarise(m = mean(porc_inv), s = sd(porc_inv))

a <- 20

dfb <- spi |> filter(country == "Spain") |> 
  select(spiyear, score_spi) |> 
  inner_join(inversiones,
             by = c("spiyear" = "anyo")) |> 
  mutate(b = score_spi/porc_inv) |> 
  group_by(grupo) |> 
  summarise(b = mean(b))

b <- setNames(dfb$b, dfb$grupo)

escenario_spi <- spi |> filter(country != "World") |> 
  select(spicountrycode, spiyear, score_spi) |> 
  mutate(inv_inf = (score_spi - rnorm(n(), 
                                      a, 1))/rnorm(n(), 
                                                   b[["INFRAESTRUCTURAS"]],
                                                   0.1*b[["INFRAESTRUCTURAS"]]),
         inv_tur = (score_spi - rnorm(n(), 
                                      a, 1))/rnorm(n(), 
                                                   b[["TURISMO"]],
                                                   0.1*b[["TURISMO"]]),
         inv_san = (score_spi - rnorm(n(), 
                                      a, 1))/rnorm(n(), 
                                                   b[["SANIDAD"]],
                                                   0.1*b[["SANIDAD"]])) |> 
  select(-score_spi)

write_csv(escenario_spi, "ESCENARIO_INVERSIONES_PAISES.csv")

## ESCENARIOS INVERSIONES CM

x_inv <- inversiones |> 
  group_by(anyo) |> 
  mutate(total_anyo = sum(inversion),
         porc_inv = round(100*(inversion/total_anyo), 2)) |> 
  ungroup() |> 
  pivot_wider(id_cols = anyo, names_from = "grupo",
              values_from = "porc_inv") |> 
  rename(spiyear = anyo,
         inv_inf = INFRAESTRUCTURAS,
         inv_tur = TURISMO,
         inv_san = SANIDAD) |> 
  select(-RESTO) |> 
  slice_tail(n = 1)

x_escen <- x_inv |> bind_rows(x_inv) |> bind_rows(x_inv) |> bind_rows(x_inv) |>  
  mutate(spiyear = 2022:2025) |> 
  mutate(across(inv_inf:inv_tur, ~.x*1.1^c(0:3)))

write_csv(x_escen, "ESCENARIO_INVERSIONES_REGION.csv")


Rows: 24 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): grupo
dbl (2): anyo, inversion

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2364 Columns: 81
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): country, spicountrycode, status
dbl (78): rank_score_spi, spiyear, score_spi, score_bhn, score_fow, score_op...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 81 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): id_var, name_var, role, Dimension, Component, Unit of measurement, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column typ